# Laboratorio: Modelos del lenguaje con RNNs

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d8/El_ingenioso_hidalgo_don_Quijote_de_la_Mancha.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano antiguo de El Ingenioso Hidalgo Don Quijote de la Mancha, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro de Don Quijote, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [1]:
import numpy as np 
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

path = keras.utils.get_file(
    fname="don_quijote.txt", 
    origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io"
)

Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [2]:
## TU CÓDIGO AQUÍ
text=open(path,encoding="utf-8").read().lower()

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [3]:
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])

Longitud del texto: 2071198
capítulo primero. que trata de la condición y ejercicio del famoso hidalgo
don quijote de la mancha


en un lugar de la mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. una olla de algo más


Eliminamos caracteres especiales

In [4]:
texto=""
for letra in text:
  if not letra in "?¿.,'¡!()-_{}\"[]«»:;\n":
    texto+=letra
text=texto

 ## Creamos un diccionario con las letras diferentes

In [5]:
letras = texto
letras_unicas = set(letras)
word_index = {}
k=0
for letra in letras_unicas:
    word_index[letra] = k
    k+=1
word_index # muestra el diccionario 

{'p': 0,
 'j': 1,
 '3': 2,
 'á': 3,
 'o': 4,
 'í': 5,
 ' ': 6,
 'ï': 7,
 'ù': 8,
 'y': 9,
 'g': 10,
 'm': 11,
 'i': 12,
 'e': 13,
 '0': 14,
 'ú': 15,
 '4': 16,
 'ó': 17,
 'q': 18,
 'f': 19,
 'u': 20,
 'ñ': 21,
 'w': 22,
 'h': 23,
 't': 24,
 'a': 25,
 '2': 26,
 'n': 27,
 '7': 28,
 'r': 29,
 'x': 30,
 'ü': 31,
 'c': 32,
 'v': 33,
 'z': 34,
 'à': 35,
 '1': 36,
 'é': 37,
 '6': 38,
 'd': 39,
 'b': 40,
 's': 41,
 '5': 42,
 'l': 43}

In [6]:
len(letras_unicas)

44

In [7]:
# Perform reverse word lookup 
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

####  Obtención de secuencias de entrada y palabra a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y los correspondientes palabras a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH palabras y la siguiente a predecir. Una vez hecho, desplazarse una palabra a la izquierda y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y los caracteres a predecir en una variable ***next_words***.

Por ejemplo, si el texto fuera "Don Quijote de La Mancha" y SEQ_LENGTH fuese 5, tendríamos

* *sequences* = ["Don Quijote de La Mancha", "Quijote de La Mancha de", "de La Mancha de cuyo", "La Mancha de cuyo nombre", "Mancha de cuyo nombre no", "de cuyo nombre no quiero"]
* *next_words* = ['de', 'cuyo', 'nombre', 'no', 'quiero', 'acordarme']

In [8]:
# Definimos el tamaño de las secuencias. Puedes dejar este valor por defecto.
SEQ_LENGTH = 10
step=1 # Para que la siguiente sentencia esté desplazada una palabra la izquierda.
sequences = []
next_words = []

## TU CÓDIGO AQUÍ

Indicar el tamaño del training set que acabamos de generar.

In [9]:
## TU CÓDIGO AQUÍ
for i in range(0,len(letras)-SEQ_LENGTH, step):
  sequences.append(letras[i:i+SEQ_LENGTH])
  next_words.append(letras[i+SEQ_LENGTH])

In [10]:
sequences[17]

'que trata '

In [11]:
next_words[17]

'd'

Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [12]:
MAX_SEQUENCES = len(sequences)

sequences, next_words = np.array(sequences), np.array(next_words)
sequences, next_words = list(sequences[:MAX_SEQUENCES]), list(next_words[:MAX_SEQUENCES])

print(len(sequences))

1968243


In [13]:
sequences[27]

'de la cond'

In [14]:
next_words[27]

'i'

# Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar one-hot encoding para nuestras palabras. Por ejemplo, si tuviéramos 4 letras las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1)

In [15]:
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, 44))
y = np.zeros((NUM_SEQUENCES, 44))

In [16]:
for i in range(NUM_SEQUENCES):
  for k,j in enumerate(sequences[i]):
    X[i][k][word_index[j]]=1
    
    
for k in range(NUM_SEQUENCES):
  y[k][word_index[next_words[k]]] = 1

# X

In [17]:
X[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

# y

In [18]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# 3. Definición del modelo y entrenamiento
Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una LSTM con 128 unidades internas. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el softmax que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [19]:
## TU CÓDIGO AQUÍ
vocabulary_size=len(letras_unicas)
model=Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH,44), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1024,activation='relu'))
model.add(Dense(44,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           88576     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1024)              132096    
                                                                 
 dense_1 (Dense)             (None, 44)                45100     
                                                                 
Total params: 397,356
Trainable params: 397,356
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
optimizer='adam'
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [21]:
history=model.fit(X,y, validation_split=0.05, batch_size=256, epochs=20,shuffle=True).history

Epoch 1/20
7305/7305 [==============================] - 474s 64ms/step - loss: 1.6741 - accuracy: 0.4828 - val_loss: 1.4447 - val_accuracy: 0.5534
Epoch 2/20
7305/7305 [==============================] - 481s 66ms/step - loss: 1.3724 - accuracy: 0.5717 - val_loss: 1.3654 - val_accuracy: 0.5763
Epoch 3/20
7305/7305 [==============================] - 484s 66ms/step - loss: 1.3089 - accuracy: 0.5892 - val_loss: 1.3324 - val_accuracy: 0.5870
Epoch 4/20
7305/7305 [==============================] - 475s 65ms/step - loss: 1.2736 - accuracy: 0.5992 - val_loss: 1.3119 - val_accuracy: 0.5919
Epoch 5/20
7305/7305 [==============================] - 471s 65ms/step - loss: 1.2492 - accuracy: 0.6057 - val_loss: 1.3044 - val_accuracy: 0.5961
Epoch 6/20
7305/7305 [==============================] - 467s 64ms/step - loss: 1.2311 - accuracy: 0.6110 - val_loss: 1.2928 - val_accuracy: 0.5986
Epoch 7/20
7305/7305 [==============================] - 474s 65ms/step - loss: 1.2162 - accuracy: 0.6149 - val_loss: 1

In [30]:
import joblib # Para salvar el modelo
joblib.dump(model,'modelo_LSTM.pkl')

['modelo_LSTM.pkl']

In [28]:
a = 0
cont = 20

for i in range(cont):
    X_pred = np.zeros((1, SEQ_LENGTH, 44))
    for j, k in enumerate(X[i]):
        X_pred[0,j] = k
    pred = model.predict(X_pred, batch_size = 32, verbose = 0)
    if (reverse_word_index[np.argmax(pred)] == reverse_word_index[np.argmax(y[i])]):
        print('-----TRUE-----')
        a+=1
    else:
        print('-----FALSE-----')
    print(f'Letra predicha: \"{reverse_word_index[np.argmax(pred)]}\"')
    print(f'Letra: {reverse_word_index[np.argmax(y[i])]}')


print(f'Aciertos {a}/{cont}')

-----FALSE-----
Letra predicha: "i"
Letra: r
-----TRUE-----
Letra predicha: "i"
Letra: i
-----TRUE-----
Letra predicha: "m"
Letra: m
-----TRUE-----
Letra predicha: "e"
Letra: e
-----TRUE-----
Letra predicha: "r"
Letra: r
-----TRUE-----
Letra predicha: "o"
Letra: o
-----TRUE-----
Letra predicha: " "
Letra:  
-----TRUE-----
Letra predicha: "q"
Letra: q
-----TRUE-----
Letra predicha: "u"
Letra: u
-----TRUE-----
Letra predicha: "e"
Letra: e
-----TRUE-----
Letra predicha: " "
Letra:  
-----FALSE-----
Letra predicha: "e"
Letra: t
-----FALSE-----
Letra predicha: "e"
Letra: r
-----TRUE-----
Letra predicha: "a"
Letra: a
-----TRUE-----
Letra predicha: "t"
Letra: t
-----TRUE-----
Letra predicha: "a"
Letra: a
-----TRUE-----
Letra predicha: " "
Letra:  
-----TRUE-----
Letra predicha: "d"
Letra: d
-----TRUE-----
Letra predicha: "e"
Letra: e
-----TRUE-----
Letra predicha: " "
Letra:  
Aciertos 17/20


# Predicción para una frase

In [31]:
X_pred = np.zeros((1, SEQ_LENGTH, vocabulary_size))
final = ''.join(sequences[0])
print(final)

for i, j in enumerate(X[0]):
    X_pred[0,i]=j

for i in range(30):
    newLeter = np.zeros(vocabulary_size)
    prediccion = model.predict(X_pred, batch_size=32, verbose=0)
    X_pred = np.delete(X_pred, 0,1)
    newLeter[np.argmax(prediccion)] = 1
    X_pred = np.append(X_pred, [[newLeter]] , axis=1)
    final += reverse_word_index[np.argmax(prediccion)]
    
print(final)

capítulo p
capítulo pi que estaba de la caballería 
